<a href="https://colab.research.google.com/github/lstate/X-SPELLS-V2/blob/main/experiments/anchor_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import re
import string

import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

!pip install anchor_exp
from anchor import anchor_text
import time

In [17]:
def cleanText(var):
    # replace punctuation with spaces
    var = re.sub('[{}]'.format(string.punctuation), " ", var)
    # remove double spaces
    var = re.sub(r'\s+', " ", var)
    # put in lower case
    var = var.lower().split()
    # remove words that are smaller than 3 characters
    var = [w for w in var if len(w) >= 3]
    var = " ".join(var)
    return var

# Removes 'rt' from all input data
def my_clean(text):
    text = text.lower().split()
    text = [w for w in text]
    text = " ".join(text)
    text = re.sub(r"rt", "", text)
    return text

# Removes 'rt' from all input data
# Removes emojis from all input data
def YOUTUBE_my_clean(text):
    text = text.lower().split()
    text = [w for w in text]
    text = " ".join(text)
    text = re.sub(r"rt", "", text)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = re.sub(emoji_pattern, '', text)
    return text


def strip_links(text):
    link_regex = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')
    return text


def strip_all_entities(text):
    entity_prefixes = ['@', '#']
    for separator in string.punctuation:
        if separator not in entity_prefixes:
            text = text.replace(separator, ' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


def preProcessing(strings):
    clean_tweet_texts = []
    for string in strings:
        clean_tweet_texts.append(my_clean(strip_all_entities(strip_links(string))))
        # clean_tweet_texts.append(my_clean(string))
    return clean_tweet_texts

def YOUTUBE_preProcessing(strings):
    clean_tweet_texts = []
    for string in strings:
        clean_tweet_texts.append(YOUTUBE_my_clean(strip_all_entities(strip_links(string))))
        # clean_tweet_texts.append(my_clean(string))
    return clean_tweet_texts

def load_dataset(dataset):
    if dataset == "polarity":
        df = pd.read_csv('https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/' + dataset_name + '_tweets.csv', encoding='utf-8')
        X = df['tweet'].values
        y = df['class'].values

    elif dataset == "hate":
        df = pd.read_csv('https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/' + dataset_name + '_tweets.csv', encoding='utf-8')
        # Removing the offensive comments, keeping only neutral and hatespeech,
        # and convert the class value from 2 to 1 for simplification purposes
        df = df[df['class'] != 1]
        X = df['tweet'].values
        y = df['class'].apply(lambda x: 1 if x == 2 else 0).values

    elif dataset == "liar":
        df_train = pd.read_csv("https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/liar_dataset/train.tsv", encoding='utf-8', sep='\t')
        df_test = pd.read_csv("https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/liar_dataset/test.tsv", encoding='utf-8', sep='\t')
        df_val = pd.read_csv("https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/liar_dataset/valid.tsv", encoding='utf-8', sep='\t')

        mapping = {'pants-fire': 0,
                   'false': 0,
                   'barely-true': 0,
                   'half-true': 1,
                   'mostly-true': 1,
                   'true': 1}

        df_train.iloc[:, 1] = df_train.iloc[:, 1].apply(lambda x: mapping[x])
        df_test.iloc[:, 1] = df_test.iloc[:, 1].apply(lambda x: mapping[x])
        df_val.iloc[:, 1] = df_val.iloc[:, 1].apply(lambda x: mapping[x])

        # Removing middle columns
        df_train = df_train[df_train.iloc[:, 1] != 2]
        df_test = df_test[df_test.iloc[:, 1] != 2]
        df_val = df_val[df_val.iloc[:, 1] != 2]

        X_train = df_train.iloc[:, 2].values
        y_train = df_train.iloc[:, 1].values
        X_test = df_test.iloc[:, 2].values
        y_test = df_test.iloc[:, 1].values
        X_val = df_val.iloc[:, 2].values
        y_val = df_val.iloc[:, 1].values

        Xtt = np.append(X_train, X_test)
        ytt = np.append(y_train, y_test)
        X = np.append(Xtt, X_val)
        y = np.append(ytt, y_val)

    elif dataset == 'question':
        df_train = pd.read_csv(("https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/question_dataset/question_train.txt"),
                               encoding='ISO-8859-1', sep=':',
                               error_bad_lines=False, header=None)
        df_test = pd.read_csv(("https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/question_dataset/question_test.txt"),
                              encoding='ISO-8859-1', sep=':',
                              error_bad_lines=False, header=None)

        def remove_first_word(string):
            return string.partition(' ')[2]

        df_train.iloc[:, 1] = df_train.iloc[:, 1].apply(remove_first_word)
        df_test.iloc[:, 1] = df_test.iloc[:, 1].apply(remove_first_word)

        X_train = df_train.iloc[:, 1].values
        y_train = df_train.iloc[:, 0].values
        X_test = df_test.iloc[:, 1].values
        y_test = df_test.iloc[:, 0].values

        X_train = preProcessing(X_train)
        X_test = preProcessing(X_test)

        X = np.append(X_train, X_test)
        y = np.append(y_train, y_test)

        # Which class to define as 0 depends on the distribution of data.
        # We pick the class with the largest number of instances.
        mapping = {'DESC': 1,
                   'ENTY': 0,
                   'ABBR': 1,
                   'HUM': 1,
                   'NUM': 1,
                   'LOC': 1}

        df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: mapping[x])
        df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: mapping[x])

        X_train = df_train.iloc[:, 1].values
        y_train = df_train.iloc[:, 0].values
        X_test = df_test.iloc[:, 1].values
        y_test = df_test.iloc[:, 0].values

        X = np.append(X_train, X_test)
        y = np.append(y_train, y_test)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.25)

    X_test = preProcessing(X_test)

    return X_test, y_test

def YOUTUBE_get_text_data():
    df = pd.read_csv('https://raw.githubusercontent.com/lstate/X-SPELLS-V2/main/data/YouTube-Spam-Collection-v1/youtube.csv', encoding='utf-8')

    X = df["CONTENT"].values
    y = df["CLASS"].values
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.25)
    
    X_test = YOUTUBE_preProcessing(X_test)
    
    # delete x/y where there is no more content after preprocessing or we have more than 140 characters (e.g. comment was only an url)
    
    indx = []
    for i in range(len(X_test)):
        if len(X_test[i]) == 0:
            indx.append(i)
        elif len(X_test[i]) > 140:
            indx.append(i)     
    X_test = np.delete(X_test, indx, 0)
    y_test = np.delete(y_test, indx, 0)
    
    return X_test, y_test

In [ ]:
!pip install spacy && python -m spacy download en_core_web_sm
!pip install torch transformers spacy && python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')

# Hate Speech

In [19]:
dataset_name = 'hate'
data, labels = load_dataset(dataset_name)
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=0.25, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer()
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)

c = sklearn.ensemble.RandomForestClassifier()
c.fit(train_vectors, train_labels)

preds = c.predict(test_vectors)

print(classification_report(test_labels, preds))
print("The accuracy score is {:.2%}".format(accuracy_score(test_labels, preds)))

def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

              precision    recall  f1-score   support

           0       0.91      0.70      0.79        87
           1       0.91      0.98      0.94       263

    accuracy                           0.91       350
   macro avg       0.91      0.84      0.87       350
weighted avg       0.91      0.91      0.90       350

The accuracy score is 90.86%


## Without using BERT

In [20]:
explainer = anchor_text.AnchorText(nlp, ['hate-speech', 'neutral'], use_unk_distribution=True)

np.random.seed(1)
text = 'fat ass hoe holding up the machine'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Text: fat ass hoe holding up the machine
Prediction: hate-speech

Anchor: ass
Precision: 1.00

Examples where anchor applies and model predicts hate-speech:

UNK ass UNK UNK up UNK machine
UNK ass hoe UNK UNK UNK UNK
fat ass UNK UNK UNK UNK UNK
fat ass hoe UNK up UNK machine
UNK ass UNK holding up the UNK
UNK ass hoe UNK up UNK UNK
UNK ass hoe UNK up the UNK
UNK ass UNK UNK UNK UNK machine
UNK ass hoe holding UNK the machine
fat ass UNK UNK UNK the UNK

Examples where anchor applies and model predicts neutral:




## Using BERT

In [21]:
explainer = anchor_text.AnchorText(nlp, ['hate-speech', 'neutral'], use_unk_distribution=False)

np.random.seed(1)
text = 'fat ass hoe holding up the machine'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Time: 58.357847452163696
Text: fat ass hoe holding up the machine
Prediction: hate-speech

Anchor: ass
Precision: 1.00

Examples where anchor applies and model predicts hate-speech:

[ ass ##ame verb ##urg ##u ##rg
thou ass ##ail him under the minute
# ass ##am ##pa $ # #
con ass ##ion # # # #
federal ass ##yrian ##20 # lo ##ndon
On ass ##oc ##12 — the funk
further ass ##im ##where is the species
1 ass ##unta ##pped up info mixtape
\ ass $ t ##w ##ool <
< ass / > d \ text

Examples where anchor applies and model predicts neutral:




# Polarity

In [22]:
dataset_name = 'polarity'
data, labels = load_dataset(dataset_name)
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=0.25, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer()
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)

c = sklearn.ensemble.RandomForestClassifier()
c.fit(train_vectors, train_labels)

preds = c.predict(test_vectors)

print(classification_report(test_labels, preds))
print("The accuracy score is {:.2%}".format(accuracy_score(test_labels, preds)))

def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

              precision    recall  f1-score   support

           0       0.65      0.71      0.68       339
           1       0.67      0.61      0.64       328

    accuracy                           0.66       667
   macro avg       0.66      0.66      0.66       667
weighted avg       0.66      0.66      0.66       667

The accuracy score is 65.97%


## Without using BERT

In [23]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=True)

np.random.seed(1)
text = 'you can practically hear george orwell turning over'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Text: you can practically hear george orwell turning over
Prediction: negative

Anchor: over
Precision: 1.00

Examples where anchor applies and model predicts negative:

you can UNK UNK george orwell turning over
you can practically UNK george orwell turning over
you can practically UNK UNK orwell UNK over
you UNK practically UNK george orwell UNK over
UNK UNK practically UNK george orwell turning over
UNK can UNK hear george UNK UNK over
you UNK UNK hear george orwell UNK over
UNK can practically hear UNK UNK turning over
UNK can practically UNK UNK UNK turning over
you UNK UNK UNK UNK orwell turning over

Examples where anchor applies and model predicts positive:




## Using BERT

In [24]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=False)

np.random.seed(1)
text = 'you can practically hear george orwell turning over'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Time: 128.07074093818665
Text: you can practically hear george orwell turning over
Prediction: negative

Anchor: hear
Precision: 1.00

Examples where anchor applies and model predicts negative:

ve - ve hear pro ##met ##her !
i can barely hear any s ##hrill ##ia
hence can also hear the cry ##k ·
does thy foe hear thy foe cry ?
we can hardly hear full disco disco ##graphy
you can now hear this sweet dragon roar
you can always hear voice commands here !
com ##place @ hear @ @ seat .
: unless we hear : w ##ows ##here
< where you hear < poem > ॥

Examples where anchor applies and model predicts positive:




# Youtube

In [25]:
dataset_name = 'youtube'
data, labels = YOUTUBE_get_text_data()
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=0.25, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer()
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)

c = sklearn.ensemble.RandomForestClassifier()
c.fit(train_vectors, train_labels)

preds = c.predict(test_vectors)

print(classification_report(test_labels, preds))
print("The accuracy score is {:.2%}".format(accuracy_score(test_labels, preds)))

def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92        55
           1       0.97      0.80      0.88        45

    accuracy                           0.90       100
   macro avg       0.92      0.89      0.90       100
weighted avg       0.91      0.90      0.90       100

The accuracy score is 90.00%


## Without using BERT

In [26]:
explainer = anchor_text.AnchorText(nlp, ['no spam', 'spam'], use_unk_distribution=True)

np.random.seed(1)
text = 'check out this video on youtube'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Text: check out this video on youtube
Prediction: spam

Anchor: on
Precision: 1.00

Examples where anchor applies and model predicts spam:

check out UNK UNK on UNK
check out this UNK on youtube
UNK UNK this video on UNK
check out this UNK on UNK
UNK out this UNK on youtube
UNK UNK UNK UNK on UNK
UNK out UNK video on youtube
check out this UNK on UNK
UNK UNK this video on UNK
UNK UNK UNK UNK on UNK

Examples where anchor applies and model predicts no spam:




## Using BERT

In [27]:
explainer = anchor_text.AnchorText(nlp, ['no spam', 'spam'], use_unk_distribution=False)

np.random.seed(1)
text = 'check out this video on youtube'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Time: 66.74689674377441
Text: check out this video on youtube
Prediction: spam

Anchor: on
Precision: 0.97

Examples where anchor applies and model predicts spam:

click button " click on "
here you gotta get on it
let build this load on :
how going to click on ?
check all this data on this
( out sun ##day on )
only 16 wrestlers got on retirement
check ##out ##ping # on !
< i > impact on conservation
put out her fork on chocolate

Examples where anchor applies and model predicts no spam:

Who pass this reference on induction
the big store went on dark
For ##cing this information on :
credits from this video on :


# Liar

In [28]:
dataset_name = 'liar'
data, labels = load_dataset(dataset_name)
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=0.25, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer()
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)

c = sklearn.ensemble.RandomForestClassifier()
c.fit(train_vectors, train_labels)

preds = c.predict(test_vectors)

print(classification_report(test_labels, preds))
print("The accuracy score is {:.2%}".format(accuracy_score(test_labels, preds)))

def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

              precision    recall  f1-score   support

           0       0.62      0.34      0.44       375
           1       0.58      0.82      0.68       425

    accuracy                           0.59       800
   macro avg       0.60      0.58      0.56       800
weighted avg       0.60      0.59      0.57       800

The accuracy score is 59.25%


## Without using BERT

In [29]:
explainer = anchor_text.AnchorText(nlp, ['fake news', 'real news'], use_unk_distribution=True)

np.random.seed(1)
text = 'why would our president close the embassy to the vatican hopefully it is not retribution for catholic organizations opposing obamacare'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Text: why would our president close the embassy to the vatican hopefully it is not retribution for catholic organizations opposing obamacare
Prediction: fake news

Anchor: president
Precision: 1.00

Examples where anchor applies and model predicts fake news:

why UNK UNK president close the embassy to the vatican hopefully UNK is not retribution for UNK organizations UNK obamacare
UNK would our president UNK the UNK UNK the vatican hopefully it UNK not retribution for catholic UNK opposing UNK
UNK UNK our president close the embassy UNK UNK vatican hopefully it UNK UNK UNK UNK catholic UNK UNK obamacare
why would our president close the embassy to UNK vatican hopefully it UNK UNK retribution for UNK UNK opposing obamacare
why would our president UNK UNK UNK to UNK vatican UNK it UNK not retribution UNK UNK UNK opposing obamacare
why would UNK president UNK UNK embassy UNK the vatican UNK it is not UNK UNK UNK organizations UNK UNK
UNK would our president close UNK embassy UNK the vatic

## Using BERT

In [30]:
explainer = anchor_text.AnchorText(nlp, ['fake news', 'real news'], use_unk_distribution=False)

np.random.seed(1)
text = 'why would our president close the embassy to the vatican hopefully it is not retribution for catholic organizations opposing obamacare'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Time: 20182.012706518173
Text: why would our president close the embassy to the vatican hopefully it is not retribution for catholic organizations opposing obamacare
Prediction: fake news

Anchor: president AND obamacare AND why AND is AND retribution
Precision: 0.97

Examples where anchor applies and model predicts fake news:

why us vice president ni ##gel ##ato asks the government allegedly responsible is seeking retributi
why - vice president ta ##urus ##ea # # # # # is seeking retribution for su ##icidal # obamacare
why one assigned president close investigative duty to seek un ##safe living is unlawful retributio
why suicide include president na ##bu ##10 # # # # # is ##o retribution terrorist person is made ob
why president vice president ng ##azi ##ma ##fia # # # # is seeking retribution for co ##lum ##bi o
why critics believe president ta ##my ##cap # # # # trust is my retribution ? nominee grand ##i oba
why ##rin ##c president # # # # # # # this is a retribution for some miss

# Question

In [31]:
dataset_name = 'question'
data, labels = load_dataset(dataset_name)
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=0.25, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

vectorizer = TfidfVectorizer()
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)

c = sklearn.ensemble.RandomForestClassifier()
c.fit(train_vectors, train_labels)

preds = c.predict(test_vectors)

print(classification_report(test_labels, preds))
print("The accuracy score is {:.2%}".format(accuracy_score(test_labels, preds)))

def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

b'Skipping line 205: expected 2 fields, saw 3\nSkipping line 258: expected 2 fields, saw 3\nSkipping line 287: expected 2 fields, saw 3\nSkipping line 298: expected 2 fields, saw 4\nSkipping line 371: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 418: expected 2 fields, saw 3\nSkipping line 476: expected 2 fields, saw 3\nSkipping line 526: expected 2 fields, saw 3\nSkipping line 590: expected 2 fields, saw 3\nSkipping line 594: expected 2 fields, saw 3\nSkipping line 695: expected 2 fields, saw 3\nSkipping line 793: expected 2 fields, saw 3\nSkipping line 861: expected 2 fields, saw 3\nSkipping line 877: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 1003: expected 2 fields, saw 3\nSkipping line 1113: expected 2 fields, saw 3\nSkipping line 1306: expected 2 fields, saw 3\nSkipping line 1406: expected 2 fields, saw 3\nSkipping line 1411: expected 2 fields, saw 3\nSkipping line 1700: expected 2 fields, saw 3\nSk

              precision    recall  f1-score   support

           0       0.80      0.24      0.37        83
           1       0.82      0.98      0.89       286

    accuracy                           0.82       369
   macro avg       0.81      0.61      0.63       369
weighted avg       0.81      0.82      0.77       369

The accuracy score is 81.57%


## Without using BERT

In [32]:
explainer = anchor_text.AnchorText(nlp, ['entity', 'all other classes'], use_unk_distribution=True)

np.random.seed(1)
text = 'what is money made of'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Text: what is money made of
Prediction: entity

Anchor: of AND made AND what
Precision: 1.00

Examples where anchor applies and model predicts entity:

what UNK money made of
what UNK UNK made of
what is UNK made of
what is money made of
what is money made of
what UNK money made of
what is UNK made of
what UNK money made of
what UNK UNK made of
what UNK money made of

Examples where anchor applies and model predicts all other classes:

what UNK UNK UNK of
what UNK UNK UNK of
what is UNK UNK of
what UNK UNK UNK of
UNK UNK money UNK of
UNK is money UNK of
what UNK UNK UNK of
what is UNK UNK of
UNK is money UNK of
what is money UNK of


## Using BERT

In [33]:
explainer = anchor_text.AnchorText(nlp, ['entity', 'all other classes'], use_unk_distribution=False)

np.random.seed(1)
text = 'what is money made of'
pred = explainer.class_names[predict_lr([text])[0]]
alternative =  explainer.class_names[1 - predict_lr([text])[0]]
b = time.time()
exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)
print('Time: %s' % (time.time() - b))

print('Text: %s' % text)
print('Prediction: %s' % pred)
print()
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Time: 203.67003774642944
Text: what is money made of
Prediction: entity

Anchor: of AND made AND what
Precision: 1.00

Examples where anchor applies and model predicts entity:

what is currently made of
what thy skin made of
what does be made of
what is usually made of
what is she made of
what is she made of
what has been made of
what they were made of
what they are made of
what is honey made of

Examples where anchor applies and model predicts all other classes:


